In [1]:
# !pip install openai langchain
# !pip install langchain_openai

!pip install pydantic

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os

#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
from langchain_openai import AzureChatOpenAI

azurechatmodel = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)

In [2]:
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict
import json
#from pydantic import BaseModel, Field
#from typing import List


In [3]:
class Ouptut(BaseModel):
    excd: str = Field(description="excd")
    claimNotes: str = Field(description="claim Notes")

class Claim(BaseModel):
    claimId: str = Field(description="claimId")
    # excd: str = Field(description="excd")
    billType : int = Field (description="billType")
    reimbursementMethod : int = Field (description="reimbursement Method")
    duplicate :bool = Field(description="duplicate")
    denyReason : str = Field (description="Deny reason")
    reimbursementMethod : str = Field (description="Reimbursment method")
    hasAdditionalRevCodes : bool = Field (description="has  Additional Rev Codes ")
    pending :bool = Field(description="pending")

    # excd : str = Field(description= "output or expected excd")
    # claimNotes: str  = Field(description="claim Notes")


# Define your desired data structure.
class Claims(BaseModel):
    currentClaim : Claim = Field(description=" current claim")
    previousClaim : Claim = Field(description="previous Claim  ")
    output_current : Ouptut = Field(description="Ouptut for current claim")
    # output_previous : Ouptut = Field(description="Ouptut for previous claim")

class ClaimsArr(BaseModel):
    details : list[Claims]

In [25]:
# And a query intented to prompt a language model to populate the data structure.
joke_query = """
You are a QA analyst, your job is to generate test data for testing Drools rules.  Please use "Drool File" to arrive at the test data.  Test data should test all possible flows in "Drool File".
Generate testdata as JSON as mentioned in the "output" only and follow the "Instructions" to generate test cases.


Instructions :

test data should cover positive flows , negative flows and edge cases as well.
Give the output only in the "output format".
Give the output as one single json
generate 10 testcase data

Drool File:
package com.edw.queries;

unit ClaimUnitData;

import com.edw.model.Claim;
import com.edw.model.Detail;

rule NonPaymentZeroPay when
$D: /details[currentClaim.billType == 0]
then
modify($D.currentClaim) {
    setExcd("5JU"),
    setClaimNotes("Claim submitted with bill type 0 provider does not expect payment.")
};
System.out.println("NonPaymentZeroPay "+$D);
end
"""


# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=ClaimsArr)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | azurechatmodel | parser

outpt = chain.invoke({"query": joke_query})

In [27]:
json_body = json.dumps(outpt)

In [28]:
json_body

'{"details": [{"currentClaim": {"claimId": "1", "billType": 0, "reimbursementMethod": "Direct Deposit", "duplicate": false, "denyReason": "", "hasAdditionalRevCodes": true, "pending": false}, "previousClaim": {"claimId": "2", "billType": 1, "reimbursementMethod": "Check", "duplicate": true, "denyReason": "Incomplete Information", "hasAdditionalRevCodes": false, "pending": true}, "output_current": {"excd": "", "claimNotes": ""}}, {"currentClaim": {"claimId": "3", "billType": 2, "reimbursementMethod": "Check", "duplicate": false, "denyReason": "", "hasAdditionalRevCodes": true, "pending": false}, "previousClaim": {"claimId": "4", "billType": 3, "reimbursementMethod": "Direct Deposit", "duplicate": false, "denyReason": "", "hasAdditionalRevCodes": true, "pending": false}, "output_current": {"excd": "", "claimNotes": ""}}, {"currentClaim": {"claimId": "5", "billType": 4, "reimbursementMethod": "Check", "duplicate": true, "denyReason": "Invalid Documentation", "hasAdditionalRevCodes": false

In [30]:
# Define your desired data structure.
class Url(BaseModel):
    raw: str
    protocol: str
    host: List[str]
    port: str
    path: List[str]

class Language(BaseModel):
    language: str = Field(description="json")    

class Options(BaseModel):
    raw: Language

class Body(BaseModel):
    mode: str = Field(description="raw")
    #raw: str
    raw: str = Field(description= json_body)# string of details + instructions --> body of the raw should be in string --> pass this in the prompt. 
    options: Options 

class Request(BaseModel):
    method: str = Field(description="POST")
    header: List[str]
    body: Body
    url: Url

list_exec = ["pm.test(\"Status code is 200\", function () {\r","pm.response.to.have.status(200);\r","});\r","pm.test(\"excd\", function () {\r",
              "var jsonData = pm.response.json();\r",
              "pm.expect(jsonData[0].currentClaim.excd).eq(\"5JU\")\r",
              "});"]

class Script2(BaseModel):
    exec: List[str] = Field(description=list_exec)
    type: str = Field(description = "text/javascript")
    packages: Dict

class Script1(BaseModel):
    exec: List[str] = Field(description="// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);")
    type: str = Field(description = "text/javascript")
    packages: Dict

class Event(BaseModel):
    listen1: str = Field(description="prerequest")
    script1: Script1
    listen2: str = Field(description="test")
    script2: Script2

class Item(BaseModel):
    name: str = Field(description="http://localhost:8080/get-claim")
    event: List[Event]
    request: Request
    response: List


class Info(BaseModel):
    #postman_id: str
    #name: str
    #schema_alias: str
    #exporter_id: str
    postman_id: str = Field(description="6259f6b2-9049-454c-95e4-b82805bfa3f8")
    name : str = Field (description="testcollection")
    schema1: str = Field (description="https://schema.getpostman.com/json/collection/v2.1.0/collection.json")
    exporter_id : int = Field(description = "_exporter_id") 

class TestCollection(BaseModel):
    info: Info
    item: List[Item]

In [31]:
sampleinput = {
	"info": {
		"_postman_id": "6259f6b2-9049-454c-95e4-b82805bfa3f8",
		"name": "testcollection",
		"schema": "https://schema.getpostman.com/json/collection/v2.1.0/collection.json",
		"_exporter_id": "98974"
	},
	"item": [
		{
			"name": "http://localhost:8080/get-claim",
			"event": [
				{
					"listen": "prerequest",
					"script": {
						"exec": [
							"// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);"
						],
						"type": "text/javascript",
						"packages": {}
					}
				},
				{
					"listen": "test",
					"script": {
						"exec": [
							"pm.test(\"Status code is 200\", function () {\r",
							"    pm.response.to.have.status(200);\r",
							"});\r",
							"pm.test(\"excd\", function () {\r",
							"    var jsonData = pm.response.json();\r",
							"    pm.expect(jsonData[0].currentClaim.excd).eq(\"5JU\")\r",
							"});"
						],
						"type": "text/javascript",
						"packages": {}
					}
				}
			],
			"request": {
				"method": "POST",
				"header": [],
				"body": {
					"mode": "raw",
					"raw": "{\"details\":[    {  \r\n        \"currentClaim\": {\r\n            \"claimId\": \"123\",\r\n            \"billType\": 0,\r\n            \"reimbursementMethod\": \"cash\",\r\n            \"duplicate\": false,\r\n            \"denyReason\": \"\",\r\n            \"hasAdditionalRevCodes\": false,\r\n            \"pending\": true\r\n        },\r\n        \"previousClaim\": {\r\n            \"claimId\": \"456\",\r\n            \"billType\": 2,\r\n            \"reimbursementMethod\": \"check\",\r\n            \"duplicate\": true,\r\n            \"denyReason\": \"Invalid claim\",\r\n            \"hasAdditionalRevCodes\": true,\r\n            \"pending\": false\r\n        } }  \r\n      ]}",
					"options": {
						"raw": {
							"language": "json"
						}
					}
				},
				"url": {
					"raw": "http://localhost:8080/get-claim",
					"protocol": "http",
					"host": [
						"localhost"
					],
					"port": "8080",
					"path": [
						"get-claim"
					]
				}
			},
			"response": []
		},
		{
			"name": "http://localhost:8080/get-claim",
			"event": [
				{
					"listen": "prerequest",
					"script": {
						"exec": [
							"// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);"
						],
						"type": "text/javascript",
						"packages": {}
					}
				},
				{
					"listen": "test",
					"script": {
						"exec": [
							"pm.test(\"Status code is 200\", function () {\r",
							"    pm.response.to.have.status(200);\r",
							"});\r",
							"pm.test(\"excd\", function () {\r",
							"    var jsonData = pm.response.json();\r",
							"    pm.expect(jsonData[0].currentClaim.excd).eq(\"5JU\")\r",
							"});"
						],
						"type": "text/javascript",
						"packages": {}
					}
				}
			],
			"request": {
				"method": "POST",
				"header": [],
				"body": {
					"mode": "raw",
					"raw": "{\"details\":[    {  \r\n        \"currentClaim\": {\r\n            \"claimId\": \"123\",\r\n            \"billType\": 5,\r\n            \"reimbursementMethod\": \"cash\",\r\n            \"duplicate\": false,\r\n            \"denyReason\": \"\",\r\n            \"hasAdditionalRevCodes\": false,\r\n            \"pending\": true\r\n        },\r\n        \"previousClaim\": {\r\n            \"claimId\": \"456\",\r\n            \"billType\": 2,\r\n            \"reimbursementMethod\": \"check\",\r\n            \"duplicate\": true,\r\n            \"denyReason\": \"Invalid claim\",\r\n            \"hasAdditionalRevCodes\": true,\r\n            \"pending\": false\r\n        } }  \r\n      ]}",
					"options": {
						"raw": {
							"language": "json"
						}
					}
				},
				"url": {
					"raw": "http://localhost:8080/get-claim",
					"protocol": "http",
					"host": [
						"localhost"
					],
					"port": "8080",
					"path": [
						"get-claim"
					]
				}
			},
			"response": []
		}
	]
}

In [6]:
json_query = """
You are a QA analyst, your job is to generate test data for testing Drools rules.  
Please use "Drool File" to arrive at the test data.  Test data should test all possible flows in "Drool File".
Generate testdata as JSON as mentioned in the {sampleinput} format only and follow the "Instructions" to generate test cases. 

Instructions :

test data should cover positive flows , negative flows and edge cases as well.
Give the output only in the "output format".
Give the output as one single json
generate 10 testcase data

Schema:
get the schema of the json from this URL - https://schema.getpostman.com/json/collection/v2.1.0/collection.json

Drool File:
package com.edw.queries;

unit ClaimUnitData;

import com.edw.model.Claim;
import com.edw.model.Detail;

rule NonPaymentZeroPay when
$D: /details[currentClaim.billType == 0]
then
modify($D.currentClaim) {
    setExcd("5JU"),
    setClaimNotes("Claim submitted with bill type 0 provider does not expect payment.")
};
System.out.println("NonPaymentZeroPay "+$D);
end
"""

In [32]:
# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=TestCollection)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n in the input sample given",
    input_variables=["query", "sampleinput"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | azurechatmodel | parser

outpt = chain.invoke({"query": json_query, "sampleinput": sampleinput})

In [33]:
outpt

{'info': {'postman_id': '6259f6b2-9049-454c-95e4-b82805bfa3f8',
  'name': 'testcollection',
  'schema1': 'https://schema.getpostman.com/json/collection/v2.1.0/collection.json',
  'exporter_id': '_exporter_id'},
 'item': [{'name': 'http://localhost:8080/get-claim',
   'event': [{'listen1': 'prerequest',
     'script1': {'exec': ['// pm.collectionVariables.set("c_billType",data["currentClaim"]["billType"]);'],
      'type': 'text/javascript',
      'packages': {}},
     'listen2': 'test',
     'script2': {'exec': ['pm.test("Status code is 200", function () {',
       'pm.response.to.have.status(200);',
       '});',
       'pm.test("excd", function () {',
       'var jsonData = pm.response.json();',
       'pm.expect(jsonData[0].currentClaim.excd).eq("5JU")',
       '});'],
      'type': 'text/javascript',
      'packages': {}}}],
   'request': {'method': 'POST',
    'header': [],
    'body': {'mode': 'raw',
     'raw': '{"details": [{"currentClaim": {"claimId": "1", "billType": 0, "reim

In [34]:
print(json.dumps(outpt))

{"info": {"postman_id": "6259f6b2-9049-454c-95e4-b82805bfa3f8", "name": "testcollection", "schema1": "https://schema.getpostman.com/json/collection/v2.1.0/collection.json", "exporter_id": "_exporter_id"}, "item": [{"name": "http://localhost:8080/get-claim", "event": [{"listen1": "prerequest", "script1": {"exec": ["// pm.collectionVariables.set(\"c_billType\",data[\"currentClaim\"][\"billType\"]);"], "type": "text/javascript", "packages": {}}, "listen2": "test", "script2": {"exec": ["pm.test(\"Status code is 200\", function () {", "pm.response.to.have.status(200);", "});", "pm.test(\"excd\", function () {", "var jsonData = pm.response.json();", "pm.expect(jsonData[0].currentClaim.excd).eq(\"5JU\")", "});"], "type": "text/javascript", "packages": {}}}], "request": {"method": "POST", "header": [], "body": {"mode": "raw", "raw": "{\"details\": [{\"currentClaim\": {\"claimId\": \"1\", \"billType\": 0, \"reimbursementMethod\": \"Direct Deposit\", \"duplicate\": false, \"denyReason\": \"\", \